Competitors
======

There are impliementations of three state of the art methods:

**Deeplog, Loganomaly, LogRobust**

Our implimentations mainly based on three Github repositories:


https://github.com/wuyifan18/DeepLog 200+ Stars

https://github.com/donglee-afar/logdeep 170+ Stars

https://github.com/HelenGuohx/logbert published a peer-reviewed paper

Target of this notebook
=====
everyone can reproduce our experiments without any missing information. Everything for reproduce is on our thesis

Our contribution
=====
* Improve **prediction speed** on GPU for **60x**

    * originally: **more than 60 minutes** 

    * ours: **only 1 minutes**

* Simplify those repositories' scripts and made them suitable for notebook

Seed
=====

```python
# use seed 1 to ensure reproducibility:
import torch
import os
import numpy as np

seed = 1

os.environ["PL_GLOBAL_SEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
``` 

# Data
there is an identical parsed data on both repositories
* https://github.com/donglee-afar/logdeep
* https://github.com/wuyifan18/DeepLog. 

This data has perfect number of log templates, we believe this one is generate from source code analysis (**most accurate way**). So for HDFS dataset we don't conduct log parsing anymore. We directly use the data from those Github repositories   

    - num_classes = 28

# Deeplog

* Model Structure
    - num_classes = 28
    - input_size = 1 (input size of LSTM layers)
    - num_layers = 2 (number of LSTM layers)
    - hidden_size = 64 (Hidden size of LSTM layers)
    - init hidden vector and memory vector of LSTM = torch.zeros(hidden_size) 

* Training parameters
    - num_epochs = 300
    - batch_size = 2048
    - optimizer = Adam
    - learning_rate = 0.001 (pytorch Adam default learning rate) 

* Data Format
    - window_size = 10 (use 10 continuous log keys to predict next log key)

* Outlier Definition
    - if at least one log key in a session can't be correctly predicted, the session is an outlier
 


*   Same network architecture, but different embedding size for different log system. Different network objects are separately trained for different log system

# Training hyperparameters

# Outlier detection








In [ ]:
! nvidia-smi

Sun Sep 26 21:10:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Preparing


*   fix bug of pytorch transformer framework
*   Connect to Gdrive



In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#!cp /content/drive/MyDrive/openstack/functional.py /usr/local/lib/python3.7/dist-packages/torch/nn/functional.py
#after first copy, restart the notebook
from tqdm.notebook import tqdm
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import argparse
import os
from collections import Counter

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import pandas as pd
import json

import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

def seed_everything(seed=1234):
    # from https://gist.github.com/KirillVladimirov/005ec7f762293d2321385580d3dbe335
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

!cp -R -f /content/drive/MyDrive/Data_30082021/ ./data

train_path = '/content/data/HDFS/train'
test_normal_path = '/content/data/HDFS/test_normal'
test_abnormal_path = '/content/data/HDFS/test_abnormal'

Mounted at /content/drive


In [ ]:
#!cp /content/drive/MyDrive/DeepLog/data/hdfs_train /content/drive/MyDrive/Data_30082021/HDFS/train
#!cp /content/drive/MyDrive/DeepLog/data/hdfs_test_normal /content/drive/MyDrive/Data_30082021/HDFS/test_normal
#!cp /content/drive/MyDrive/DeepLog/data/hdfs_test_abnormal /content/drive/MyDrive/Data_30082021/HDFS/test_abnormal

#!cp /content/drive/MyDrive/openstack/openstack_train.txt /content/drive/MyDrive/Data_30082021/Openstack/train
#!cp /content/drive/MyDrive/openstack/openstack_test_normal.txt /content/drive/MyDrive/Data_30082021/Openstack/test_normal
#!cp /content/drive/MyDrive/openstack/openstack_test_abnormal.txt /content/drive/MyDrive/Data_30082021/Openstack/test_abnormal

#!cp /content/drive/MyDrive/hadoop/hadoop_train.txt /content/drive/MyDrive/Data_30082021/Hadoop/train
#!cp /content/drive/MyDrive/hadoop/hadoop_test_normal.txt /content/drive/MyDrive/Data_30082021/Hadoop/test_normal
#!cp /content/drive/MyDrive/hadoop/hadoop_test_abnormal.txt /content/drive/MyDrive/Data_30082021/Hadoop/test_abnormal

# Common Function

In [ ]:
def generate(path, window_size = 10,
             quantitative_features = False,
             vocab_size = 28,
             need_sessions_number = False,dedup = False,**kwargs):
    num_sessions = 0
    inputs = []
    quantitative = []
    semantic = []
    outputs = []
    session_id = []
    #default load hdfs with realtive path, other dataset need provide full path
    #name = path.split('/')[-1]

    if isinstance(path,str):
        if dedup:
            line_list = set([])
            with open(path,'r') as f:
                for line in f.readlines():
                    line_list.add(line.strip())
                line_list = list(line_list)
        else:
            with open(path,'r') as f:
                line_list = f.readlines()
    else:
        line_list = path
        if dedup:
            line_list = list(set(line_list))

    for line in line_list:
        num_sessions += 1
        line = list(map(lambda n: n-1, map(int, line.strip().split())))
        line = line + [28] * (window_size + 1 - len(line))

        for i in range(len(line) - window_size):
            ##########
            # key generation of each log sequence
            ##########
            inputs.append(line[i:i + window_size])

            if quantitative_features:
                Quantitative_pattern = [0] * vocab_size
                log_counter = Counter(line[i:i + window_size])
                for key in log_counter:
                    if key>vocab_size-1:
                        continue
                    Quantitative_pattern[key] = log_counter[key]
                quantitative.append(Quantitative_pattern)

            outputs.append(line[i + window_size])
            session_id.append(num_sessions)
    #print('Number of sessions({}): {}'.format(name, num_sessions))
    #print('Number of seqs({}): {}'.format(name, len(inputs)))
    if quantitative_features:
        dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float),
                                torch.tensor(quantitative, dtype=torch.float), 
                                torch.tensor(outputs), 
                                torch.tensor(session_id))
    else:
        dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs), torch.tensor(session_id))
    if need_sessions_number:
        return dataset,num_sessions
    else:
        return dataset


def evaluate_result(result_normal,result_abnormal,length,**kwargs):
    FP = len(result_normal)
    TP = len(result_abnormal)
    FN = length - TP
    P = 100 * TP / (TP + FP)
    R = 100 * TP / (TP + FN)
    F1 = 2 * P * R / (P + R)
    print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
    print('Finished Predicting')

    return P,R,F1


def write_result(result_list,**kwargs):
    #calculate mean and stdanard variance
    result = pd.DataFrame(result_list).describe()
    mean = result.loc['mean']
    std = result.loc['std']
    result = tuple(zip(mean,std))

    # format logging sequence
    content = {'Model_name':kwargs['Model_name'],
        'Dataset_name':kwargs['Dataset_name'],
        'dedup':kwargs['dedup'],
        'result': result}
    content = json.dumps(content)+'\n'
    print(content)
    # wirte to the file
    with open(config['wirte_result_to'],'a') as data:
        data.write(content)

------------

# Deeplog specific function

In [ ]:
# Deeplog
class Deeplog(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_keys,**kwargs):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_keys)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def train_deeplog(model, dataloader, model_dir,learning_rate,**kwargs):
    #device
    device = next(model.parameters()).device

    # Hyperparameters
    num_classes = kwargs['num_classes']
    num_epochs = kwargs['num_epochs']
    batch_size = kwargs['batch_size']
    input_size = kwargs['input_size']
    num_layers = kwargs['num_layers']
    hidden_size = kwargs['hidden_size']
    window_size = kwargs['window_size']


    # Loss and optimizer
    criterion = nn.CrossEntropyLoss(ignore_index=28)
    optimizer = optim.Adam(model.parameters(),lr = learning_rate)

    # Training loop
    start_time = time.time()
    total_step = len(dataloader)
    with tqdm(range(num_epochs)) as t:
        for epoch in t:  # Loop over the dataset multiple times
            train_loss = 0
            for step, (seq, label,_) in enumerate(dataloader):
                # Forward pass
                seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
                output = model(seq)
                loss = criterion(output, label.to(device))

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                train_loss += loss.item()
                optimizer.step()
                #scheduler.step()

            t.set_postfix({'_': 'Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step)})

    # Summary of training
    elapsed_time = time.time() - start_time
    print('elapsed_time: {:.3f}s'.format(elapsed_time))
    if not os.path.isdir(model_dir):
        os.makedirs(model_dir)
    torch.save(model.state_dict(), model_dir+'.pt')
    print('Finished Training')


def predict_deeplog(model, dataloader, window_size = 10, input_size = 1, top = 9,**kwargs):
    result = set([])
    model.eval()
    device = next(model.parameters()).device
    
    for i in tqdm(dataloader):    
        # get probabilities 
        f = model(i[0].clone().detach().view(-1, window_size, input_size).to(device))

        #check if windows are outlier
        predict = (torch.argsort(f, 1)[:,-top:] == i[1].reshape(-1,1).cuda()).any(-1).tolist()

        # outlier result
        id = i[2].tolist()
        for i in range(len(id)):
            if not predict[i]:
                result.add(id[i])
    model.train()
    return result




# original dataset

In [ ]:
with open(train_path) as data:
    train_data = list(data.readlines())
with open(test_abnormal_path) as data:
    test_abnormal_data = list(data.readlines())


In [ ]:
import json
config = {'Model_name': 'deeplog',
          'Dataset_name':'HDFS',
          'dedup':False,
          'wirte_result_to': '/content/drive/MyDrive/Final_result_30082021.txt',
                'input_size': 1,
                'num_classes':29,
                'num_keys':29,
                'hidden_size':64,
                'num_epochs': 300,
                'batch_size': 2048,
                'num_layers': 2,
                'window_size':10,
                'vocab_size': 28,
                'learning_rate': 0.001,
                'model_dir': '.',
                'top': 9}

# dataset
seq_dataset = generate(train_path,**config)
train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

normal_data = generate(test_normal_path,**config)
test_normal_dataloader = DataLoader(normal_data,batch_size=config['batch_size']*4)

abnormal,length = generate(test_abnormal_path,need_sessions_number=True,**config)
test_abnormal_dataloader = DataLoader(abnormal,batch_size=config['batch_size'])

######### part don't need to change#############################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_list = []

for seed in range(10):
    seed_everything(seed)
    #define model        
    deeplog = Deeplog(**config).to(device)

    #train
    train_deeplog(deeplog, train_dataloader, **config)

    #predict
    result_normal = predict_deeplog(deeplog,test_normal_dataloader, **config)
    result_abnormal = predict_deeplog(deeplog,test_abnormal_dataloader,**config)

    #write result
    result_list.append(evaluate_result(result_normal= result_normal,
                            result_abnormal = result_abnormal,length = length)) #(P,R,F1)
    
write_result(result_list,**config)

In [ ]:
# dirty version
with open(train_path) as data:
    train_data = list(data.readlines())
with open(test_abnormal_path) as data:
    test_abnormal_data = list(data.readlines())

import json
config = {'Model_name': 'deeplog',
          'Dataset_name':'HDFS',
          'dedup':False,
          'wirte_result_to': '/content/drive/MyDrive/Final_result_dirty.txt',
                'input_size': 1,
                'num_classes':29,
                'num_keys':29,
                'hidden_size':64,
                'num_epochs': 300,
                'batch_size': 2048,
                'num_layers': 2,
                'window_size':10,
                'vocab_size': 28,
                'learning_rate': 0.001,
                'model_dir': '.',
                'top': 9}

# dataset
#seq_dataset = generate(train_path,**config)
#train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

normal_data = generate(test_normal_path,**config)
test_normal_dataloader = DataLoader(normal_data,batch_size=config['batch_size']*4)

abnormal,length = generate(test_abnormal_path,need_sessions_number=True,**config)
test_abnormal_dataloader = DataLoader(abnormal,batch_size=config['batch_size'])

######### part don't need to change#############################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_list = []

for dirty_percent in range(8):
    seed_everything(1)

    dirty_train_set = train_data + random.sample(test_abnormal_data,int(4855*0.0071*dirty_percent))
    seq_dataset = generate(dirty_train_set,**config)
    train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

    #define model        
    deeplog = Deeplog(**config).to(device)

    #train
    train_deeplog(deeplog, train_dataloader, **config)

    #predict
    result_normal = predict_deeplog(deeplog,test_normal_dataloader, **config)
    result_abnormal = predict_deeplog(deeplog,test_abnormal_dataloader,**config)

    #write result
    result_list.append(evaluate_result(result_normal= result_normal,
                            result_abnormal = result_abnormal,length = length)) #(P,R,F1)
    
write_result(result_list,**config)
[93.756%,66.639%,64.079%,61.430%,62.392%,61.029%,58.524%,57.212%]

  0%|          | 0/300 [00:00<?, ?it/s]

elapsed_time: 193.117s
Finished Training


  0%|          | 0/642 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

false positive (FP): 832, false negative (FN): 1245, Precision: 94.935%, Recall: 92.606%, F1-measure: 93.756%
Finished Predicting


...

  0%|          | 0/300 [00:00<?, ?it/s]

elapsed_time: 202.795s
Finished Training


  0%|          | 0/642 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

false positive (FP): 1089, false negative (FN): 9655, Precision: 86.835%, Recall: 42.659%, F1-measure: 57.212%
Finished Predicting
{"Model_name": "deeplog", "Dataset_name": "HDFS", "dedup": false, "result": [[90.5592468722648, 2.365197556754479], [52.62278774201212, 16.461111152370105], [65.63258851070587, 11.742074493109046]]}



In [ ]:
a = {"Model_name": "deeplog", "Dataset_name": "HDFS", "dedup": False, "result": [[95.28155228642987, 0.6618687845711819], [91.15631310131845, 6.770232678483861], [93.04141804080375, 3.8179064853278453]]}

In [ ]:
pd.DataFrame(a['result']).round(3)

0      1
0  95.282  0.662
1  91.156  6.770
2  93.041  3.818

# dedup dataset

In [ ]:
import json
config = {'Model_name': 'deeplog',
          'Dataset_name':'HDFS',
          'dedup':True,
          'wirte_result_to': '/content/drive/MyDrive/Final_result_30082021.txt',
                'input_size': 1,
                'num_classes':29,
                'num_keys':29,
                'hidden_size':64,
                'num_epochs': 300,
                'batch_size': 2048,
                'num_layers': 2,
                'window_size':10,
                'vocab_size': 28,
                'learning_rate': 0.001,
                'model_dir': '.',
                'top': 9}

# dataset
seq_dataset = generate(train_path,**config)
train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

normal_data = generate(test_normal_path,**config)
test_normal_dataloader = DataLoader(normal_data,batch_size=config['batch_size']*4)

abnormal,length = generate(test_abnormal_path,need_sessions_number=True,**config)
test_abnormal_dataloader = DataLoader(abnormal,batch_size=config['batch_size'])

######### part don't need to change#############################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_list = []

for seed in range(10):
    seed_everything(seed)
    #define model        
    deeplog = Deeplog(**config).to(device)

    #train
    train_deeplog(deeplog, train_dataloader, **config)

    #predict
    result_normal = predict_deeplog(deeplog,test_normal_dataloader, **config)
    result_abnormal = predict_deeplog(deeplog,test_abnormal_dataloader,**config)

    #write result
    result_list.append(evaluate_result(result_normal= result_normal,
                            result_abnormal = result_abnormal,length = length)) #(P,R,F1)
    
write_result(result_list,**config)

Number of sessions(train): 839
Number of seqs(train): 11725
Number of sessions(test_normal): 14177
Number of seqs(test_normal): 241216
Number of sessions(test_abnormal): 4123
Number of seqs(test_abnormal): 80185


  0%|          | 0/300 [00:00<?, ?it/s]

elapsed_time: 48.695s
Finished Training


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/300 [00:00<?, ?it/s]

elapsed_time: 49.189s
Finished Training


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

false positive (FP): 471, false negative (FN): 276, Precision: 89.092%, Recall: 93.306%, F1-measure: 91.150%
Finished Predicting
{"Model_name": "deeplog", "Dataset_name": "HDFS", "dedup": true, "result": [[86.20750875554876, 1.787382407830311], [90.7179238418627, 1.9043627738693223], [88.39390168986563, 1.517598344242061]]}



In [ ]:
# dirty version
with open(train_path) as data:
    train_data = list(data.readlines())
with open(test_abnormal_path) as data:
    test_abnormal_data = list(data.readlines())

import json
config = {'Model_name': 'deeplog',
          'Dataset_name':'HDFS',
          'dedup':True,
          'wirte_result_to': '/content/drive/MyDrive/Final_result_dirty.txt',
                'input_size': 1,
                'num_classes':29,
                'num_keys':29,
                'hidden_size':64,
                'num_epochs': 300,
                'batch_size': 2048,
                'num_layers': 2,
                'window_size':10,
                'vocab_size': 28,
                'learning_rate': 0.001,
                'model_dir': '.',
                'top': 9}

# dataset
#seq_dataset = generate(train_path,**config)
#train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

normal_data = generate(test_normal_path,**config)
test_normal_dataloader = DataLoader(normal_data,batch_size=config['batch_size']*4)

abnormal,length = generate(test_abnormal_path,need_sessions_number=True,**config)
test_abnormal_dataloader = DataLoader(abnormal,batch_size=config['batch_size'])

######### part don't need to change#############################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_list = []

for dirty_percent in range(8):
    seed_everything(1)

    dirty_train_set = train_data + random.sample(test_abnormal_data,int(4855*0.0071*dirty_percent))
    seq_dataset = generate(dirty_train_set,**config)
    train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

    #define model        
    deeplog = Deeplog(**config).to(device)

    #train
    train_deeplog(deeplog, train_dataloader, **config)

    #predict
    result_normal = predict_deeplog(deeplog,test_normal_dataloader, **config)
    result_abnormal = predict_deeplog(deeplog,test_abnormal_dataloader,**config)

    #write result
    result_list.append(evaluate_result(result_normal= result_normal,
                            result_abnormal = result_abnormal,length = length)) #(P,R,F1)
    
write_result(result_list,**config)

[88.696%,59.465%,43.734%,34.695%,37.447%,28.150%,27.787%,27.612%]

  0%|          | 0/300 [00:00<?, ?it/s]

elapsed_time: 50.689s
Finished Training


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

false positive (FP): 554, false negative (FN): 396, Precision: 87.059%, Recall: 90.395%, F1-measure: 88.696%
Finished Predicting


...

  0%|          | 0/300 [00:00<?, ?it/s]

elapsed_time: 59.530s
Finished Training


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

false positive (FP): 447, false negative (FN): 3391, Precision: 62.087%, Recall: 17.754%, F1-measure: 27.612%
Finished Predicting
{"Model_name": "deeplog", "Dataset_name": "HDFS", "dedup": true, "result": [[72.89585529231678, 8.712632780324352], [33.69512490904681, 24.960323732563072], [43.448258552092376, 21.20209957753937]]}



# Loganomaly

### Model

In [ ]:
# Loganomaly
class Loganomaly(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_keys,**kwargs):
        super(Loganomaly, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm0 = nn.LSTM(input_size,
                             hidden_size,
                             num_layers,
                             batch_first=True)
        self.lstm1 = nn.LSTM(input_size,
                             hidden_size,
                             num_layers,
                             batch_first=True)
        self.fc = nn.Linear(2 * hidden_size, num_keys)

    # it is very strange in logdeep implimentation. They impliment attention_net
    # but they don't use it any where in this class

    def forward(self, features, device):
        input0, input1 = features[0], features[1]

        h0_0 = torch.zeros(self.num_layers, input0.size(0),
                           self.hidden_size).to(device)
        c0_0 = torch.zeros(self.num_layers, input0.size(0),
                           self.hidden_size).to(device)

        out0, _ = self.lstm0(input0, (h0_0, c0_0))

        h0_1 = torch.zeros(self.num_layers, input1.size(0),
                           self.hidden_size).to(device)
        c0_1 = torch.zeros(self.num_layers, input1.size(0),
                           self.hidden_size).to(device)

        out1, _ = self.lstm1(input1, (h0_1, c0_1))
        multi_out = torch.cat((out0[:, -1, :], out1[:, -1, :]), -1)
        out = self.fc(multi_out)
        return out

def train_loganomaly(model,dataloader,learning_rate,
                     lr_step, lr_decay_ratio,window_size,input_size,num_classes,
                     num_epochs,model_dir,**kwargs):
    #device
    device = next(model.parameters()).device

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr = learning_rate)
    scheduler = MultiStepLR(optimizer, milestones=lr_step, gamma=lr_decay_ratio)

    #train loops
    start_time = time.time()
    total_step = len(dataloader)
    with tqdm(range(num_epochs)) as t:
        for epoch in t:  # Loop over the dataset multiple times
            train_loss = 0
            for step, (seq, quantitative ,label,_) in enumerate(dataloader):
                # Forward pass
                seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
                quantitative = quantitative.clone().detach().view(-1, num_classes-1, input_size).to(device)
                output = model((seq, quantitative),device)
                loss = criterion(output, label.to(device))

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                train_loss += loss.item()
                optimizer.step()

            scheduler.step()

            t.set_postfix({'_': 'Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step),
                        'learning_rate': str(scheduler.get_last_lr())})

    # Summary of training
    elapsed_time = time.time() - start_time
    print('elapsed_time: {:.3f}s'.format(elapsed_time))
    if not os.path.isdir(model_dir):
        os.makedirs(model_dir)
    #torch.save(model.state_dict(), model_dir + '/' + log + '.pt')
    print('Finished Training')




def predict_loganomaly(model, test_loader,window_size,input_size,top,batch_size,**kwargs):
    result2 = set([])
    device = next(model.parameters()).device
    model.eval()
    for i in tqdm(DataLoader(test_loader,batch_size*4)):    
        # input parsing
        seq = i[0].clone().detach().view(-1, window_size, input_size).to(device)
        quantitative = i[1].clone().detach().view(-1, i[1].size(1), input_size).to(device)
        y = i[2].reshape(-1,1).to(device)
        id = i[3].tolist()

        # model prediction
        f = model((seq,quantitative),device)

        predict = (torch.argsort(f, 1)[:,-top:] == y).any(-1).tolist()

        
        for i in range(len(id)):
            if not predict[i]:
                result2.add(id[i])
    return result2

# original

In [ ]:
import json
from torch.optim.lr_scheduler import MultiStepLR
config = {'Model_name': 'Loganomaly',
          'Dataset_name':'HDFS',
          'dedup':False,
          'quantitative_features': True,
          'wirte_result_to': '/content/drive/MyDrive/Final_result_30082021.txt',
                'input_size': 1,
                'num_classes':29,
                'num_keys':29,
                'hidden_size':64,
                'num_epochs': 370,
                'batch_size': 2048,
                'num_layers': 2,
                'window_size':10,
                'vocab_size': 28,
                'learning_rate': 0.001,
                'lr_step': (300,350),
                'lr_decay_ratio': 0.1,
                'model_dir': '.',
                'top': 9}

# dataset
seq_dataset = generate(train_path,**config)
train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

normal_data,_ = generate(test_normal_path,need_sessions_number=True,**config)

abnormal,length = generate(test_abnormal_path,need_sessions_number=True,**config)

######### part don't need to change#############################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_list = []

for seed in range(10):
    seed_everything(seed)
    #define model (specify by model)        
    loganomaly = Loganomaly(**config).to(device)

    #train (specify by model)
    train_loganomaly(loganomaly, train_dataloader, **config)

    #predict (specify by model)
    result_normal = predict_loganomaly(loganomaly,normal_data, **config)
    result_abnormal = predict_loganomaly(loganomaly,abnormal,**config)

    #write result
    result_list.append(evaluate_result(result_normal= result_normal,
                            result_abnormal = result_abnormal,length = length)) #(P,R,F1)
    
write_result(result_list,**config)

Number of sessions(train): 4855
Number of seqs(train): 46575
Number of sessions(test_abnormal): 16838
Number of seqs(test_abnormal): 168890


  0%|          | 0/370 [00:00<?, ?it/s]

elapsed_time: 378.017s
Finished Training


  0%|          | 0/642 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

...

  0%|          | 0/370 [00:00<?, ?it/s]

elapsed_time: 372.590s
Finished Training


  0%|          | 0/642 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

false positive (FP): 538, false negative (FN): 507, Precision: 96.811%, Recall: 96.989%, F1-measure: 96.900%
Finished Predicting
{"Model_name": "Loganomaly", "Dataset_name": "HDFS", "dedup": false, "result": [[96.00104566419621, 0.6949258550895082], [95.10274379379973, 2.9597658417696606], [95.52335388583776, 1.3681341763739945]]}



In [ ]:
# dirty version
with open(train_path) as data:
    train_data = list(data.readlines())
with open(test_abnormal_path) as data:
    test_abnormal_data = list(data.readlines())



import json
from torch.optim.lr_scheduler import MultiStepLR
config = {'Model_name': 'Loganomaly',
          'Dataset_name':'HDFS',
          'dedup':False,
          'quantitative_features': True,
          'wirte_result_to': '/content/drive/MyDrive/Final_result_30082021.txt',
                'input_size': 1,
                'num_classes':29,
                'num_keys':29,
                'hidden_size':64,
                'num_epochs': 370,
                'batch_size': 2048,
                'num_layers': 2,
                'window_size':10,
                'vocab_size': 28,
                'learning_rate': 0.001,
                'lr_step': (300,350),
                'lr_decay_ratio': 0.1,
                'model_dir': '.',
                'top': 9}

# dataset
seq_dataset = generate(train_path,**config)
train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

normal_data,_ = generate(test_normal_path,need_sessions_number=True,**config)

abnormal,length = generate(test_abnormal_path,need_sessions_number=True,**config)

######### part don't need to change#############################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_list = []

for dirty_percent in range(8):
    seed_everything(1)

    dirty_train_set = train_data + random.sample(test_abnormal_data,int(4855*0.0071*dirty_percent))
    seq_dataset = generate(dirty_train_set,**config)
    train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

    #define model (specify by model)        
    loganomaly = Loganomaly(**config).to(device)

    #train (specify by model)
    train_loganomaly(loganomaly, train_dataloader, **config)

    #predict (specify by model)
    result_normal = predict_loganomaly(loganomaly,normal_data, **config)
    result_abnormal = predict_loganomaly(loganomaly,abnormal,**config)

    #write result
    result_list.append(evaluate_result(result_normal= result_normal,
                            result_abnormal = result_abnormal,length = length)) #(P,R,F1)
    
write_result(result_list,**config)


[95.270%,31.469%,21.207%,16.877%,15.894%,15.139%,10.705%,12.169%]

  0%|          | 0/370 [00:00<?, ?it/s]

elapsed_time: 581.409s
Finished Training


  0%|          | 0/642 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

false positive (FP): 766, false negative (FN): 824, Precision: 95.435%, Recall: 95.106%, F1-measure: 95.270%
Finished Predicting


...

  0%|          | 0/370 [00:00<?, ?it/s]

elapsed_time: 616.135s
Finished Training


  0%|          | 0/642 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

false positive (FP): 774, false negative (FN): 15697, Precision: 59.582%, Recall: 6.776%, F1-measure: 12.169%
Finished Predicting
{"Model_name": "Loganomaly", "Dataset_name": "HDFS", "dedup": false, "result": [[72.2455032203441, 12.100267560476762], [20.837540087896425, 30.305669338761174], [27.341270555343193, 28.196823236838465]]}



# deduplicate

In [ ]:
import json
config = {'Model_name': 'Loganomaly',
          'Dataset_name':'HDFS',
          'dedup':True, # dedup
          'quantitative_features': True, #loganomaly
          'wirte_result_to': '/content/drive/MyDrive/Final_result_30082021.txt',
                'input_size': 1,
                'num_classes':29,
                'num_keys':29,
                'hidden_size':64,
                'num_epochs': 370,
                'batch_size': 2048,
                'num_layers': 2,
                'window_size':10,
                'vocab_size': 28,
                'learning_rate': 0.001,
                'lr_step': (300,350),
                'lr_decay_ratio': 0.1,
                'model_dir': '.',
                'top': 9}

# dataset
seq_dataset = generate(train_path,**config)
train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

normal_data,_ = generate(test_normal_path,need_sessions_number=True,**config)

abnormal,length = generate(test_abnormal_path,need_sessions_number=True,**config)

######### part don't need to change#############################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_list = []

for seed in range(10):
    seed_everything(seed)
    #define model (specify by model)        
    loganomaly = Loganomaly(**config).to(device)

    #train (specify by model)
    train_loganomaly(loganomaly, train_dataloader, **config)

    #predict (specify by model)
    result_normal = predict_loganomaly(loganomaly,normal_data, **config)
    result_abnormal = predict_loganomaly(loganomaly,abnormal,**config)

    #write result
    result_list.append(evaluate_result(result_normal= result_normal,
                            result_abnormal = result_abnormal,length = length)) #(P,R,F1)
    
write_result(result_list,**config)

Number of sessions(train): 839
Number of seqs(train): 11725
Number of sessions(test_normal): 14177
Number of seqs(test_normal): 241216
Number of sessions(test_abnormal): 4123
Number of seqs(test_abnormal): 80185


  0%|          | 0/370 [00:00<?, ?it/s]

elapsed_time: 95.519s
Finished Training


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/370 [00:00<?, ?it/s]

elapsed_time: 96.599s
Finished Training


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

false positive (FP): 465, false negative (FN): 214, Precision: 89.369%, Recall: 94.810%, F1-measure: 92.009%
Finished Predicting
{"Model_name": "Loganomaly", "Dataset_name": "HDFS", "dedup": true, "result": [[88.54065666042746, 1.0864512717519825], [92.19015280135824, 3.27438468658023], [90.30538550880351, 1.836737273574194]]}



In [ ]:
# dirty version
with open(train_path) as data:
    train_data = list(data.readlines())
with open(test_abnormal_path) as data:
    test_abnormal_data = list(data.readlines())



import json
from torch.optim.lr_scheduler import MultiStepLR
config = {'Model_name': 'Loganomaly',
          'Dataset_name':'HDFS',
          'dedup':True,
          'quantitative_features': True,
          'wirte_result_to': '/content/drive/MyDrive/Final_result_30082021.txt',
                'input_size': 1,
                'num_classes':29,
                'num_keys':29,
                'hidden_size':64,
                'num_epochs': 370,
                'batch_size': 2048,
                'num_layers': 2,
                'window_size':10,
                'vocab_size': 28,
                'learning_rate': 0.001,
                'lr_step': (300,350),
                'lr_decay_ratio': 0.1,
                'model_dir': '.',
                'top': 9}

# dataset
seq_dataset = generate(train_path,**config)
train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

normal_data,_ = generate(test_normal_path,need_sessions_number=True,**config)

abnormal,length = generate(test_abnormal_path,need_sessions_number=True,**config)

######### part don't need to change#############################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_list = []

for dirty_percent in range(8):
    seed_everything(1)

    dirty_train_set = train_data + random.sample(test_abnormal_data,int(4855*0.0071*dirty_percent))
    seq_dataset = generate(dirty_train_set,**config)
    train_dataloader = DataLoader(seq_dataset, batch_size=config['batch_size'], shuffle=True)

    #define model (specify by model)        
    loganomaly = Loganomaly(**config).to(device)

    #train (specify by model)
    train_loganomaly(loganomaly, train_dataloader, **config)

    #predict (specify by model)
    result_normal = predict_loganomaly(loganomaly,normal_data, **config)
    result_abnormal = predict_loganomaly(loganomaly,abnormal,**config)

    #write result
    result_list.append(evaluate_result(result_normal= result_normal,
                            result_abnormal = result_abnormal,length = length)) #(P,R,F1)
    
write_result(result_list,**config)

[92.284%,52.902%,45.747%,32.634%,32.040%,27.033%,30.643%,28.013%]

  0%|          | 0/370 [00:00<?, ?it/s]

elapsed_time: 153.738s
Finished Training


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

false positive (FP): 519, false negative (FN): 146, Precision: 88.456%, Recall: 96.459%, F1-measure: 92.284%
Finished Predicting


...

  0%|          | 0/370 [00:00<?, ?it/s]

elapsed_time: 179.336s
Finished Training


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

false positive (FP): 451, false negative (FN): 3378, Precision: 62.291%, Recall: 18.069%, F1-measure: 28.013%
Finished Predicting
{"Model_name": "Loganomaly", "Dataset_name": "HDFS", "dedup": true, "result": [[70.72895398184771, 8.896210531214757], [33.434392432694644, 26.70209873941772], [42.66199996244299, 22.017711274653184]]}



# Competitor result view

## hdfs

In [ ]:
with open('/content/drive/MyDrive/Final_result_30082021.txt') as data:
    df = []
    for i in data:

        i = i.replace('false','False')
        i = i.replace('true','True')
        df.append(eval(i.strip()))

import pandas as pd
df = [i for i in df if i['Dataset_name']=='HDFS']
df = pd.DataFrame(df)
x = pd.DataFrame(df.result.to_list())
def format(x):
    mean = round(float(x[0]),3)
    std = round(float(x[1]),3)
    return str(mean)+'%'+'('+str(std)+')'

for i in range(3):
    x[i] = x[i].apply(format)

df.merge(x,left_index=True,right_index=True)

Model_name Dataset_name  ...               1               2
0     deeplog         HDFS  ...   91.156%(6.77)  93.041%(3.818)
1     deeplog         HDFS  ...  90.718%(1.904)  88.394%(1.518)
2  Loganomaly         HDFS  ...   95.103%(2.96)  95.523%(1.368)
3  Loganomaly         HDFS  ...   92.19%(3.274)  90.305%(1.837)

[4 rows x 7 columns]

In [ ]:
df

Empty DataFrame
Columns: []
Index: []

# openstack and hadoop

In [ ]:

with open('/content/drive/MyDrive/Final_result_2.txt') as data:
    df = []
    for i in data:

        i = i.replace('false','False')
        i = i.replace('true','True')
        df.append(eval(i.strip()))

import pandas as pd
df = pd.DataFrame(df)
x = pd.DataFrame(df.result.to_list())
def format(x):
    mean = round(float(x[0]),3)
    std = round(float(x[1]),3)
    return str(mean)+'%'+'('+str(std)+')'

for i in range(3):
    x[i] = x[i].apply(format)

df = df.merge(x,left_index=True,right_index=True)

In [ ]:
df[df.dedup == True]

Model_name Dataset_name  dedup  ...               0             1               2
1     deeplog    Openstack   True  ...  20.644%(0.054)  65.789%(0.0)  31.427%(0.062)
3     deeplog       Hadoop   True  ...    89.583%(0.0)   100.0%(0.0)    94.505%(0.0)
5  Loganomaly       Hadoop   True  ...    89.583%(0.0)   100.0%(0.0)    94.505%(0.0)
6  Loganomaly    Openstack   True  ...  20.627%(0.076)  65.789%(0.0)  31.407%(0.088)
7  Loganomaly    Openstack   True  ...  20.644%(0.054)  65.789%(0.0)  31.427%(0.062)

[5 rows x 7 columns]

In [ ]:

with open('/content/drive/MyDrive/Final_result_30082021.txt') as data:
    df = []
    for i in data:

        i = i.replace('false','False')
        i = i.replace('true','True')
        df.append(eval(i.strip()))

import pandas as pd
df = pd.DataFrame(df)
x = pd.DataFrame(df.result.to_list())
def format(x):
    mean = round(float(x[0]),3)
    std = round(float(x[1]),3)
    return str(mean)+'%'+'('+str(std)+')'

for i in range(3):
    x[i] = x[i].apply(format)

df = df.merge(x,left_index=True,right_index=True)

In [ ]:
df[df.dedup == True]

Model_name Dataset_name  ...               1               2
1     deeplog    Openstack  ...     2.632%(0.0)    4.74%(0.047)
5     deeplog         HDFS  ...  90.718%(1.904)  88.394%(1.518)
8  Loganomaly         HDFS  ...   92.19%(3.274)  90.305%(1.837)

[3 rows x 7 columns]

In [ ]:
ori =[((0.13168449197860962, 0.9949494949494949, 0.23258559622195984, 0.11267605633802817), 0.11934122009784572), ((0.13411078717201166, 0.9292929292929293, 0.23439490445859873, 0.08695652173913043), 0.1710996895673247), ((0.13095238095238096, 1.0, 0.23157894736842108, 0.08888888888888889), 0.13033620387458), ((0.13086582947785855, 1.0, 0.23144360023378138, 0.0), 0.08876536202555266), ((0.1322930800542741, 0.9848484848484849, 0.23325358851674644, 0.1095890410958904), 0.14409058273084116), ((0.13112582781456952, 1.0, 0.23185011709601872, 0.0958904109589041), 0.13896944819454446), ((0.13095238095238096, 1.0, 0.23157894736842108, 0.044444444444444446), 0.14038667335485028), ((0.13095238095238096, 1.0, 0.23157894736842108, 0.022222222222222223), 0.17384548648759865), ((0.13095238095238096, 1.0, 0.23157894736842108, 0.06666666666666667), 0.14144018704085154)]
dedup=[((0.22289156626506024, 0.9736842105263158, 0.3627450980392157, 0.11267605633802817), 0.20692834480472147), ((0.2236024844720497, 0.9473684210526315, 0.36180904522613067, 0.08450704225352113), 0.23077434789156592), ((0.2122905027932961, 1.0, 0.35023041474654376, 0.08888888888888889), 0.20216630098298147), ((0.2111111111111111, 1.0, 0.3486238532110092, 0.0), 0.15286471989779005), ((0.21818181818181817, 0.9473684210526315, 0.354679802955665, 0.1095890410958904), 0.2342604793788172), ((0.21468926553672316, 1.0, 0.35348837209302325, 0.0958904109589041), 0.24554419116599016), ((0.2122905027932961, 1.0, 0.35023041474654376, 0.044444444444444446), 0.23195059349029393), ((0.2122905027932961, 1.0, 0.35023041474654376, 0.022222222222222223), 0.22932520949752375), ((0.2122905027932961, 1.0, 0.35023041474654376, 0.06666666666666667), 0.22463287548690963)]

In [ ]:
data = pd.DataFrame([i[0] for i in ori]).iloc[:,:3]
data = data*100
data = data.describe().loc[['mean','std'],:].T
data.round(3).apply(lambda x: str(x['mean'])+'%'+'('+str(x['std'])+')',axis =1)

0    13.154%(0.107)
1     98.99%(2.328)
2     23.22%(0.102)
dtype: object

In [ ]:
import pandas as pd
data = pd.DataFrame([i[0] for i in dedup]).iloc[:,:3]
data = data*100
data = data.describe().loc[['mean','std'],:].T
data.round(3).apply(lambda x: str(x['mean'])+'%'+'('+str(x['std'])+')',axis =1)

0    21.552%(0.486)
1    98.538%(2.321)
2    35.359%(0.527)
dtype: object

# Tranformerlog result view

In [ ]:
#Transformerlog
ori = [((0.8926292278238672, 0.9968523577622045, 0.9418663374670333, 0.013333333333333334), 0.9530997668278303), ((0.8938175621705096, 0.9968523577622045, 0.942527444759525, 0.013333333333333334), 0.9460032491306225), ((0.9540529808837711, 0.9988715999524884, 0.9759480082397656, 0.010638297872340425), 0.9851614973004826), ((0.9418513543765391, 0.9994654947143367, 0.9698034921915519, 0.009615384615384616), 0.9788704472348081), ((0.975361136999068, 0.9944767787148118, 0.9848262071399165, 0.013157894736842105), 0.9907797976200667), ((0.9744722916787811, 0.9952488419052143, 0.984750991626267, 0.013333333333333334), 0.9911489334878285), ((0.9734811118203447, 0.9963178524765411, 0.9847671039887294, 0.013333333333333334), 0.9881060146777536), ((0.9633501838235294, 0.9959615156194322, 0.9793844536588214, 0.013333333333333334), 0.9862720565352662), ((0.9766859007985079, 0.9951894524290296, 0.9858508604206501, 0.013157894736842105), 0.9905296669271829), ((0.9695516524150681, 0.9966147998574653, 0.9828969718268611, 0.013333333333333334), 0.989324412056491), ((0.967678633267921, 0.9957239577146929, 0.9815009951996253, 0.013157894736842105), 0.9868275622878634)]
dedup = [((0.8597676874340021, 0.987387824399709, 0.9191691126665161, 0.013333333333333334), 0.9204850422787185), ((0.8610406091370558, 0.987387824399709, 0.9198960569427183, 0.013333333333333334), 0.8801158337770718), ((0.8998026748520062, 0.9953917050691244, 0.9451865499769692, 0.010638297872340425), 0.938607228905519), ((0.8635600335852225, 0.9978171234537958, 0.925846742432767, 0.009615384615384616), 0.9294935847651833), ((0.9370206832442483, 0.9779286926994907, 0.9570377403275577, 0.013157894736842105), 0.9584286912326576), ((0.9247656071346901, 0.9808391947610963, 0.9519774011299434, 0.013333333333333334), 0.9537345979001772), ((0.9219246482069905, 0.9852049478535048, 0.9525149489975379, 0.013333333333333334), 0.9514549673620645), ((0.9193109700815957, 0.983749696822702, 0.9504393673110723, 0.013333333333333334), 0.9300237859533994), ((0.9300988733042078, 0.9810817365995634, 0.9549102927289895, 0.013157894736842105), 0.9550942773783404), ((0.928326866012326, 0.9864176570458404, 0.9564910630291628, 0.013333333333333334), 0.9554433636941574), ((0.9184043517679057, 0.9827795294688334, 0.949502050380785, 0.013157894736842105), 0.9269409613510792)]

In [ ]:
data = pd.DataFrame([i[0] for i in dedup]).iloc[:,:3]
data = data*100
data = data.describe().loc[['mean','std'],:].T
data.round(3).apply(lambda x: str(x['mean'])+'%'+'('+str(x['std'])+')',axis =1)

0    90.582%(2.996)
1      98.6%(0.603)
2    94.391%(1.477)
dtype: object

In [ ]:
data = pd.DataFrame([i[0] for i in ori]).iloc[:,:3]
data = data*100
data = data.describe().loc[['mean','std'],:].T
data.round(3).apply(lambda x: str(x['mean'])+'%'+'('+str(x['std'])+')',axis =1)

0    95.299%(3.132)
1    99.651%(0.152)
2    97.401%(1.643)
dtype: object

In [ ]:
df[df.Dataset_name == 'HDFS']

Model_name Dataset_name  ...               1               2
2     deeplog         HDFS  ...   91.156%(6.77)  93.041%(3.818)
5     deeplog         HDFS  ...  90.718%(1.904)  88.394%(1.518)
7  Loganomaly         HDFS  ...   95.103%(2.96)  95.523%(1.368)
8  Loganomaly         HDFS  ...   92.19%(3.274)  90.305%(1.837)

[4 rows x 7 columns]

### Parameter

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_classes = 29
num_epochs = 370
batch_size = 2048
input_size = 1
model_dir = 'model'
log = 'Adam_batch_size={}_epoch={}'.format(str(batch_size), str(num_epochs))
num_layers = 2
hidden_size = 64
window_size = 10

# 
learning_rate = 0.001
accumulation_step = 1
lr_step = (300,350)
lr_decay_ratio =0.1

num_candidates = 9

from torch.optim.lr_scheduler import MultiStepLR

model = loganomaly(input_size, hidden_size, num_layers, num_classes).to(device)

### Training

In [ ]:
device = next(model.parameters()).device

#logging
#writer = SummaryWriter(log_dir='/content/drive/MyDrive/DeepLog/log2' + log)

#training data
seq_dataset = generate('hdfs_train',quantitative_features = True)
dataloader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

# Loss and optimizer and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)
scheduler = MultiStepLR(optimizer, milestones=lr_step, gamma=lr_decay_ratio)

# Training loop
start_time = time.time()
total_step = len(dataloader)
with tqdm(range(num_epochs)) as t:
    for epoch in t:  # Loop over the dataset multiple times
        train_loss = 0
        for step, (seq, quantitative ,label,_) in enumerate(dataloader):
            # Forward pass
            seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
            quantitative = quantitative.clone().detach().view(-1, num_classes-1, input_size).to(device)
            output = model((seq, quantitative),device)
            loss = criterion(output, label.to(device))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            train_loss += loss.item()
            optimizer.step()

        scheduler.step()

        t.set_postfix({'_': 'Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step),
                      'learning_rate': str(scheduler.get_last_lr())})

# Summary of training
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + log + '.pt')
print('Finished Training')

In [ ]:
test_normal_loader,_ = generate('hdfs_test_normal',
                                quantitative_features= True,need_sessions_number=True)

result1 = predict(model,test_normal_loader)
FP = len(result1)


test_abnormal_loader,length = generate('hdfs_test_abnormal',
                                       need_sessions_number=True,quantitative_features = True)

result2 = predict(model,test_abnormal_loader)

TP = len(result2)
print(TP)
FN = length - TP
P = 100 * TP / (TP + FP)
R = 100 * TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
print('Finished Predicting')

In [ ]:
random.seed(1234) #keep reproducbility
random_seeds = [random.randint(0,100) for i in range(10)]
rerun_results_loganomaly = []

In [ ]:
#rerun with different seed

seq_dataset = generate('hdfs_train',quantitative_features = True)

test_normal_loader,_ = generate('hdfs_test_normal',
                            quantitative_features= True,need_sessions_number=True)

test_abnormal_loader,length = generate('hdfs_test_abnormal',
                                    need_sessions_number=True,quantitative_features = True)

rerun_results_loganomaly = []

for seed in range(10):
    seed_everything(seed)
    #################
    # hyperparameter
    #################
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Hyperparameters
    num_classes = 29
    num_epochs = 370
    batch_size = 2048
    input_size = 1
    model_dir = 'model'
    log = 'Adam_batch_size={}_epoch={}'.format(str(batch_size), str(num_epochs))
    num_layers = 2
    hidden_size = 64
    window_size = 10

    # 
    learning_rate = 0.001
    accumulation_step = 1
    lr_step = (300,350)
    lr_decay_ratio =0.1

    num_candidates = 9

    from torch.optim.lr_scheduler import MultiStepLR

    model = loganomaly(input_size, hidden_size, num_layers, num_classes).to(device)


    device = next(model.parameters()).device

    #################
    # train
    #################

    #training data
    #seq_dataset = generate('hdfs_train',quantitative_features = True)
    dataloader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

    # Loss and optimizer and scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr = learning_rate)
    scheduler = MultiStepLR(optimizer, milestones=lr_step, gamma=lr_decay_ratio)

    # Training loop
    start_time = time.time()
    total_step = len(dataloader)
    with tqdm(range(num_epochs)) as t:
        for epoch in t:  # Loop over the dataset multiple times
            train_loss = 0
            for step, (seq, quantitative ,label,_) in enumerate(dataloader):
                # Forward pass
                seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
                quantitative = quantitative.clone().detach().view(-1, num_classes-1, input_size).to(device)
                output = model((seq, quantitative),device)
                loss = criterion(output, label.to(device))

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                train_loss += loss.item()
                optimizer.step()

            scheduler.step()

            t.set_postfix({'_': 'Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step),
                        'learning_rate': str(scheduler.get_last_lr())})

    # Summary of training
    elapsed_time = time.time() - start_time
    print('elapsed_time: {:.3f}s'.format(elapsed_time))
    if not os.path.isdir(model_dir):
        os.makedirs(model_dir)
    torch.save(model.state_dict(), model_dir + '/' + log + '.pt')
    print('Finished Training')

    #################
    # evaluation
    #################

    #test_normal_loader,_ = generate('hdfs_test_normal',
    #                            quantitative_features= True,need_sessions_number=True)

    result1 = predict(model,test_normal_loader)
    FP = len(result1)


    #test_abnormal_loader,length = generate('hdfs_test_abnormal',
    #                                    need_sessions_number=True,quantitative_features = True)

    result2 = predict(model,test_abnormal_loader)

    TP = len(result2)
    print(TP)
    FN = length - TP
    P = 100 * TP / (TP + FP)
    R = 100 * TP / (TP + FN)
    F1 = 2 * P * R / (P + R)
    print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
    print('Finished Predicting')


    rerun_results_loganomaly.append((P,R,F1))

print(rerun_results_loganomaly)

In [ ]:
print(random_seeds)
print(rerun_results_loganomaly)

In [ ]:
import pandas as pd
def print_P_R_F1(a):
    P = [i[0] for i in a]
    R = [i[1] for i in a]
    F1 = [i[2] for i in a]
    return pd.DataFrame({'P':P,'R':R,'F1':F1})

In [ ]:
print_P_R_F1(rerun_results_loganomaly).describe()

# without duplication

## deeplog

In [ ]:
def generate_nodup(path, window_size = 10,
             quantitative_features = False,
             vocab_size = 28,
             need_sessions_number = False):
    num_sessions = 0
    inputs = []
    quantitative = []
    semantic = []
    outputs = []
    session_id = []
    #default load hdfs with realtive path, other dataset need provide full path
    if path[0]!='/':
        path = '/content/drive/MyDrive/DeepLog/data/' + path
    name = path.split('/')[-1]
    deduplicate = set([])
    with open(path,'r') as f:
        for line in f.readlines():
            deduplicate.add(line.strip())

    for line in list(deduplicate):
        num_sessions += 1
        line = list(map(lambda n: n-1, map(int, line.strip().split())))
        line = line + [28] * (window_size + 1 - len(line))

        for i in range(len(line) - window_size):
            ##########
            # key generation of each log sequence
            ##########
            inputs.append(line[i:i + window_size])

            if quantitative_features:
                Quantitative_pattern = [0] * vocab_size
                log_counter = Counter(line[i:i + window_size])
                for key in log_counter:
                    if key>vocab_size-1:
                        continue
                    Quantitative_pattern[key] = log_counter[key]
                quantitative.append(Quantitative_pattern)

            outputs.append(line[i + window_size])
            session_id.append(num_sessions)
    print('Number of sessions({}): {}'.format(name, num_sessions))
    print('Number of seqs({}): {}'.format(name, len(inputs)))
    if quantitative_features:
        dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float),
                                torch.tensor(quantitative, dtype=torch.float), 
                                torch.tensor(outputs), 
                                torch.tensor(session_id))
    else:
        dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs), torch.tensor(session_id))
    if need_sessions_number:
        return dataset,num_sessions
    else:
        return dataset

In [ ]:
#seed everything
#seed_everything(1234)

###############
# model define
###############
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')
# Hyperparameters
num_classes = 29
num_epochs = 300
batch_size = 2048
input_size = 1
num_layers = 2
hidden_size = 64
window_size = 10

seed_everything(1234)
deeplog = Deeplog(input_size, hidden_size, num_layers, num_classes).to(device)

#########
# data  #
#########
#train data:
seq_dataset = generate_nodup('hdfs_train')
train_dataloader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True)
saving_path = '/content/drive/MyDrive/DeepLog/data/hdfs_deeplog'

#predict data:
#normal
normal_data = generate_nodup('hdfs_test_normal')
test_normal_dataloader = DataLoader(normal_data,2048)
#abnormal
abnormal,length = generate_nodup('hdfs_test_abnormal',need_sessions_number=True)
test_abnormal_dataloader = DataLoader(abnormal,2048)

##############
# training   #
##############
train_deeplog(deeplog, train_dataloader, saving_path, learning_rate= 0.001)

################
# predicting   #
################
# predict normal
result1 = predict_deeplog(deeplog,test_normal_dataloader,
                          window_size= window_size, 
                          input_size = input_size, top = 9)
# predict abnormal
result2 = predict_deeplog(deeplog,test_abnormal_dataloader,
                          window_size= window_size, 
                          input_size = input_size, top = 9)

################
# print result #
################
FP = len(result1)
TP = len(result2)
FN = length - TP
P = 100 * TP / (TP + FP)
R = 100 * TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
print('Finished Predicting')

## ten times rerun

In [ ]:
#seed everything
#seed_everything(1234)
rerun_results = []
###############
# model define
###############
for seed in range(10):
    # rerun with different seed
    seed_everything(seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #device = torch.device('cpu')
    # Hyperparameters
    num_classes = 29
    num_epochs = 300
    batch_size = 2048
    input_size = 1
    num_layers = 2
    hidden_size = 64
    window_size = 10

    deeplog = Deeplog(input_size, hidden_size, num_layers, num_classes).to(device)

    #########
    # data  #
    #########
    #train data:
    seq_dataset = generate_nodup('hdfs_train')
    train_dataloader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True)
    saving_path = '/content/drive/MyDrive/DeepLog/data/hdfs_deeplog'

    #predict data:
    #normal
    normal_data = generate_nodup('hdfs_test_normal')
    test_normal_dataloader = DataLoader(normal_data,2048)
    #abnormal
    abnormal,length = generate_nodup('hdfs_test_abnormal',need_sessions_number=True)
    test_abnormal_dataloader = DataLoader(abnormal,2048)

    ##############
    # training   #
    ##############
    train_deeplog(deeplog, train_dataloader, saving_path, learning_rate= 0.001)

    ################
    # predicting   #
    ################
    # predict normal
    result1 = predict_deeplog(deeplog,test_normal_dataloader,
                            window_size= window_size, 
                            input_size = input_size, top = 9)
    # predict abnormal
    result2 = predict_deeplog(deeplog,test_abnormal_dataloader,
                            window_size= window_size, 
                            input_size = input_size, top = 9)

    ################
    # print result #
    ################
    FP = len(result1)
    TP = len(result2)
    FN = length - TP
    P = 100 * TP / (TP + FP)
    R = 100 * TP / (TP + FN)
    F1 = 2 * P * R / (P + R)
    print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
    print('Finished Predicting')

    rerun_results.append((P,R,F1))

print(random_seeds)
print('#####')
print(rerun_results)

In [ ]:
import pandas as pd
def a(list_):
    m = []
    for i in list_:
        if isinstance(i[0],tuple):
            m.append(i[0])
        else:
            m.append(i)
    return pd.DataFrame(m,columns = ['P','R','F1'])
(a(rerun_results)/100.0).describe()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_classes = 29
num_epochs = 370
batch_size = 2048
input_size = 1
model_dir = 'model'
log = 'Adam_batch_size={}_epoch={}'.format(str(batch_size), str(num_epochs))
num_layers = 2
hidden_size = 64
window_size = 10

# 
learning_rate = 0.001
accumulation_step = 1
lr_step = (300,350)
lr_decay_ratio =0.1

num_candidates = 9

from torch.optim.lr_scheduler import MultiStepLR

model = loganomaly(input_size, hidden_size, num_layers, num_classes).to(device)


device = next(model.parameters()).device

#logging
#writer = SummaryWriter(log_dir='/content/drive/MyDrive/DeepLog/log2' + log)

#training data
seq_dataset = generate_nodup('hdfs_train',quantitative_features = True)
dataloader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

# Loss and optimizer and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)
scheduler = MultiStepLR(optimizer, milestones=lr_step, gamma=lr_decay_ratio)

# Training loop
start_time = time.time()
total_step = len(dataloader)
with tqdm(range(num_epochs)) as t:
    for epoch in t:  # Loop over the dataset multiple times
        train_loss = 0
        for step, (seq, quantitative ,label,_) in enumerate(dataloader):
            # Forward pass
            seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
            quantitative = quantitative.clone().detach().view(-1, num_classes-1, input_size).to(device)
            output = model((seq, quantitative),device)
            loss = criterion(output, label.to(device))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            train_loss += loss.item()
            optimizer.step()

        scheduler.step()

        t.set_postfix({'_': 'Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step),
                      'learning_rate': str(scheduler.get_last_lr())})

# Summary of training
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + log + '.pt')
print('Finished Training')

test_normal_loader,_ = generate_nodup('hdfs_test_normal',
                                quantitative_features= True,need_sessions_number=True)

result1 = predict(model,test_normal_loader)
FP = len(result1)


test_abnormal_loader,length = generate_nodup('hdfs_test_abnormal',
                                       need_sessions_number=True,quantitative_features = True)

result2 = predict(model,test_abnormal_loader)

TP = len(result2)
print(TP)
FN = length - TP
P = 100 * TP / (TP + FP)
R = 100 * TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
print('Finished Predicting')

In [ ]:
#rerun with different seed

seq_dataset = generate_nodup('hdfs_train',quantitative_features = True)

test_normal_loader,_ = generate_nodup('hdfs_test_normal',
                            quantitative_features= True,need_sessions_number=True)

test_abnormal_loader,length = generate_nodup('hdfs_test_abnormal',
                                    need_sessions_number=True,quantitative_features = True)

rerun_results_loganomaly = []

for seed in range(10):
    seed_everything(seed)
    #################
    # hyperparameter
    #################
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Hyperparameters
    num_classes = 29
    num_epochs = 370
    batch_size = 2048
    input_size = 1
    model_dir = 'model'
    log = 'Adam_batch_size={}_epoch={}'.format(str(batch_size), str(num_epochs))
    num_layers = 2
    hidden_size = 64
    window_size = 10

    # 
    learning_rate = 0.001
    accumulation_step = 1
    lr_step = (300,350)
    lr_decay_ratio =0.1

    num_candidates = 9

    from torch.optim.lr_scheduler import MultiStepLR

    model = loganomaly(input_size, hidden_size, num_layers, num_classes).to(device)


    device = next(model.parameters()).device

    #################
    # train
    #################

    #training data
    #seq_dataset = generate('hdfs_train',quantitative_features = True)
    dataloader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

    # Loss and optimizer and scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr = learning_rate)
    scheduler = MultiStepLR(optimizer, milestones=lr_step, gamma=lr_decay_ratio)

    # Training loop
    start_time = time.time()
    total_step = len(dataloader)
    with tqdm(range(num_epochs)) as t:
        for epoch in t:  # Loop over the dataset multiple times
            train_loss = 0
            for step, (seq, quantitative ,label,_) in enumerate(dataloader):
                # Forward pass
                seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
                quantitative = quantitative.clone().detach().view(-1, num_classes-1, input_size).to(device)
                output = model((seq, quantitative),device)
                loss = criterion(output, label.to(device))

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                train_loss += loss.item()
                optimizer.step()

            scheduler.step()

            t.set_postfix({'_': 'Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step),
                        'learning_rate': str(scheduler.get_last_lr())})

    # Summary of training
    elapsed_time = time.time() - start_time
    print('elapsed_time: {:.3f}s'.format(elapsed_time))
    if not os.path.isdir(model_dir):
        os.makedirs(model_dir)
    torch.save(model.state_dict(), model_dir + '/' + log + '.pt')
    print('Finished Training')

    #################
    # evaluation
    #################

    #test_normal_loader,_ = generate('hdfs_test_normal',
    #                            quantitative_features= True,need_sessions_number=True)

    result1 = predict(model,test_normal_loader)
    FP = len(result1)


    #test_abnormal_loader,length = generate('hdfs_test_abnormal',
    #                                    need_sessions_number=True,quantitative_features = True)

    result2 = predict(model,test_abnormal_loader)

    TP = len(result2)
    print(TP)
    FN = length - TP
    P = 100 * TP / (TP + FP)
    R = 100 * TP / (TP + FN)
    F1 = 2 * P * R / (P + R)
    print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
    print('Finished Predicting')


    rerun_results_loganomaly.append((P,R,F1))

print(rerun_results_loganomaly)

In [ ]:
(a(rerun_results_loganomaly)/100.0).describe()